In [1]:
import torch
from PIL import Image
from torchvision import transforms as T

import os
import glob
import shutil
import pandas as pd

# HyperParameters

In [2]:
batch_size = 32
image_path = "data/image_path/"
sure_image_path = "data/sure image/"
device = "cuda"
sure_vector_path = "data/sure image/sure_image_vector.dict"
label_file = "data/df.csv"
target_path = "data/dataset/"
reserved_suffix = ["jpeg", "png"]

"""
最小距离为0.75的要至少占比50%
最小距离为0.55的要至少占比100%
"""
rule_list = [
    {"percentage": .5, "min_distance": .75},
    {"percentage": 1., "min_distance": .55},
]

# Model

In [3]:
transform = T.Compose([
    T.Resize(364),
    T.CenterCrop((364, 364)),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [4]:
model = torch.hub.load('dinov2', 'dinov2_vitl14_lc', source="local", pretrained=True)
model.to(device)
model.eval();

/home/zhy/NanShan_animal/dinov2/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/zhy/NanShan_animal/dinov2/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/zhy/NanShan_animal/dinov2/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


# Sure Image Vector

In [5]:
classes = sorted([
    path.split("/")[-1]
    for path in glob.iglob(f"{sure_image_path}/*")
    if os.path.isdir(path)
])

In [6]:
sure_image_vector = {}
if sure_vector_path is None:
    with torch.no_grad():
        for cls in classes:
            cache = []
            img_path = glob.glob(f"{sure_image_path}/{cls}/*")
            for i in range(0, len(img_path), batch_size):
                img = [
                    transform(Image.open(path).convert("RGB"))
                    for path in img_path[i: i + batch_size]
                ]
                x = torch.stack(img).to(device)
                cache.append(model(x).cpu())

            sure_image_vector[cls] = torch.cat(cache)
    torch.save(sure_image_vector, "data/sure image/sure_image_vector.dict")
else:
    sure_image_vector = torch.load(sure_vector_path)

# Similarigy Distance

In [7]:
def cosine_similarigy(A, B):
    """ 计算向量间的余弦值
    Args:
        A (Tensor): shape->(N, D)
        B (Tensor): shape->(M, D)
        
    Return: shape->(N, M)
    """
    A = A[:, None, :]
    B = B[None, :, :]
    return (
        (A * B).sum(dim=-1) / 
        (A.pow(2).sum(dim=-1).sqrt() * B.pow(2).sum(dim=-1).sqrt())
    )

def filter_distance_image(vector, rule=None):
    """ 根据规则筛选向量
    Args:
        vector (Tensor): 距离向量(M, N), N为目标图片数量, M为源图片数量
        rule (List[dict]): 筛选规则, 距离向量应满足所有规则要求才能拷贝到所对应目录,
            dict应含有`percentage` (所占目标向量总数的百分比)和`min_distance` (最小距离大小)
    """
    if not rule:
        raise "未指定任何规则信息"
    
    target_count = vector.size(1)
    cache_vector = torch.ones(vector.size(0))
    for r in rule:
        cache_vector *= (
            (
                (vector > r["min_distance"]).sum(-1) /
                target_count
            ) >= r["percentage"]
        )
        
    return cache_vector

In [8]:
# 删除不明后缀名
df = pd.read_csv(label_file)
df["file_suffix"] = df.file_name.str.split(".").str[-1]
df = df[df["file_suffix"].isin(reserved_suffix)].reset_index(drop=True)

In [9]:
with torch.no_grad():
    for cls in classes:
        target_vector = sure_image_vector[cls]
        cache = []
        img_path = df.query(f"name == '{cls}'").file_name.to_list()
        for i in range(0, len(img_path), batch_size):
            img = [
                transform(Image.open(os.path.join(image_path, path)).convert("RGB"))
                for path in img_path[i: i + batch_size]
            ]
            x = torch.stack(img).to(device)
            cache.append(model(x).cpu())

        cls_vector = torch.cat(cache)
        distance = cosine_similarigy(cls_vector, target_vector)
        if not os.path.exists(os.path.join(target_path, cls)):
            os.mkdir(os.path.join(target_path, cls))
        
        filter_vector = filter_distance_image(distance, rule_list)
        for flag, path in zip(filter_vector, img_path):
            if flag == 1:
                shutil.copy(
                    os.path.join(image_path, path),
                    os.path.join(target_path, cls, path)
                )
        print(f"{cls}\t\t源图片数量：{distance.size(0)}\t 筛选后图片数量: {int(filter_vector.sum())}")

东方蝙蝠		源图片数量：971	 筛选后图片数量: 35
乌鸫		源图片数量：873	 筛选后图片数量: 838
凤头鹰		源图片数量：922	 筛选后图片数量: 806
北社鼠		源图片数量：38	 筛选后图片数量: 5
北红尾鸲		源图片数量：914	 筛选后图片数量: 892
华南兔		源图片数量：518	 筛选后图片数量: 21
喜鹊		源图片数量：888	 筛选后图片数量: 488
家燕		源图片数量：864	 筛选后图片数量: 806
小䴙䴘		源图片数量：789	 筛选后图片数量: 772
小星头啄木鸟		源图片数量：529	 筛选后图片数量: 279
山斑鸠		源图片数量：889	 筛选后图片数量: 859
戴胜		源图片数量：967	 筛选后图片数量: 962
斑姬啄木鸟		源图片数量：879	 筛选后图片数量: 825
普通翠鸟		源图片数量：900	 筛选后图片数量: 889
松雀鹰		源图片数量：910	 筛选后图片数量: 699
林雕		源图片数量：783	 筛选后图片数量: 435
树莺		源图片数量：864	 筛选后图片数量: 815
棕背伯劳		源图片数量：944	 筛选后图片数量: 936
灰喜鹊		源图片数量：937	 筛选后图片数量: 891
灰树鹊		源图片数量：472	 筛选后图片数量: 292
灰脸鵟鹰		源图片数量：684	 筛选后图片数量: 444
珠颈斑鸠		源图片数量：899	 筛选后图片数量: 864
画眉		源图片数量：882	 筛选后图片数量: 714
白头鹎		源图片数量：861	 筛选后图片数量: 847
白鹭		源图片数量：895	 筛选后图片数量: 878
竹鸡		源图片数量：888	 筛选后图片数量: 639
红嘴蓝鹊		源图片数量：897	 筛选后图片数量: 875
红头长尾山雀		源图片数量：910	 筛选后图片数量: 850
红胁蓝尾鸲		源图片数量：892	 筛选后图片数量: 849
红腹松鼠		源图片数量：901	 筛选后图片数量: 339
红隼		源图片数量：899	 筛选后图片数量: 873
绿头鸭		源图片数量：838	 筛选后图片数量: 745
褐家鼠		源图片数量：806	 筛选后图片数量: 420
赤腹鹰		源图片数量：867	 筛选后图片数量: 604


/home/zhy/miniconda3/envs/torch2.0/lib/python3.9/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


远东刺猬		源图片数量：402	 筛选后图片数量: 194
野猪		源图片数量：869	 筛选后图片数量: 641
银喉长尾山雀		源图片数量：914	 筛选后图片数量: 885
长尾缝叶莺		源图片数量：742	 筛选后图片数量: 495
雀鹰		源图片数量：938	 筛选后图片数量: 806
雉鸡		源图片数量：837	 筛选后图片数量: 763
鹊鸲		源图片数量：896	 筛选后图片数量: 843
麻雀		源图片数量：873	 筛选后图片数量: 685
黄喉鹀		源图片数量：934	 筛选后图片数量: 852
黄眉柳莺		源图片数量：873	 筛选后图片数量: 612
黄鼬		源图片数量：916	 筛选后图片数量: 183
黑水鸡		源图片数量：866	 筛选后图片数量: 854
黑翅短脚鹎		源图片数量：334	 筛选后图片数量: 53
黑脸噪鹛		源图片数量：788	 筛选后图片数量: 508
黑鸢		源图片数量：873	 筛选后图片数量: 555
